<a href="https://colab.research.google.com/github/Z1TH1Z/demo-repo/blob/main/MFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import necessary Libraries**

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from scipy.stats import skew, kurtosis

# **Neural Network-based recommender class**

In [ ]:
class NeuralNetworkModelRecommender:
  # Initialize basic variables and model containers
  def __init__(self, epochs=50, batch_size=32, data_type='tabular'):
    self.epochs = epochs
    self.batch_size = batch_size
    self.data_type = data_type
    self.scaler = StandardScaler()
    self.label_encoder = LabelEncoder()
    self.ml_models = {
      'classifiers': {
        'Random Forest Classifier': RandomForestClassifier(),
        'Gradient Boosting Classifier': GradientBoostingClassifier(),
        'SVM Classifier': SVC(probability=True),
        'Logistic Regression': LogisticRegression(max_iter=1000)
      },
      'regressors': {
        'Random Forest Regressor': RandomForestRegressor(),
        'Gradient Boosting Regressor': GradientBoostingRegressor(),
        'SVM Regressor': SVR(),
        'Ridge Regression': Ridge()
      }
    }
    self.metadata_model = None      # Neural network to predict best model
    self.trained_model_dict = None  # Trained models as references

  # Fit and extract features from input data based on its type
  def fit(self, data):
    if self.data_type == 'tabular':
      if hasattr(data, "iloc"):
        X, y = data.iloc[:, :-1].values, data.iloc[:, -1].values

      elif isinstance(data, (tuple, list)) and len(data) == 2:
        X, y = data

      else:
        data = np.array(data)
        X, y = data[:, :-1], data[:, -1]

      if X.ndim > 2:
        X = X.reshape(X.shape[0], -1)

      X = self.scaler.fit_transform(X)
      self.X, self.y = X, y

    elif self.data_type == 'image':
      # Preprocess image data and extract features via CNN + PCA
      X, y = data
      X = X.astype('float32') / 255.0
      X, y = shuffle(X, y)
      sample_size = min(1000, len(X))   # Sample subset for quicker computation
      X = X[:sample_size]
      y = y[:sample_size]

      input_shape = X.shape[1:]
      feature_extractor = keras.Sequential([
        keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
      ])
      X = feature_extractor.predict(X, verbose=0)

      pca = PCA(n_components=min(50, X.shape[1]))
      X = pca.fit_transform(X)

    elif self.data_type == 'pixel':
      # Flatten pixel data and scale
      X, y = data
      X = X.astype('float32') / 255.0
      X = X.reshape(X.shape[0], -1)
      X = self.scaler.fit_transform(X)

    elif self.data_type == 'text':
      # Convert text to TF-IDF features
      X, y = data
      self.vectorizer = TfidfVectorizer(max_features=1000)
      X = self.vectorizer.fit_transform(X).toarray()

      if X.ndim > 2:
        X = X.reshape(X.shape[0], -1)

      X = self.scaler.fit_transform(X)

    else:
      raise ValueError("Invalid data_type. Use 'tabular', 'image', or 'pixel'.")

    # Label encoding the target
    y = self.label_encoder.fit_transform(y)

    # Extract metadata to help in the meta-model learning process
    metadata = self._extract_metadata(X, y)
    problem_type = self._infer_problem_type(y)
    model_dict = self.ml_models[problem_type]
    self.trained_model_dict = model_dict

    X_meta, y_meta = [], []

    for i, (model_name, model) in enumerate(model_dict.items()):
      try:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
        model.fit(X_train, y_train)
        score = model.score(X_test, y_test)
        X_meta.append(metadata)
        y_meta.append(i)
      except Exception as e:
        if len(np.unique(y)) < 2:
          print(f"Skipping {model_name}: only one class present in the data")
          continue

    X_meta = np.array(X_meta)
    y_meta = keras.utils.to_categorical(y_meta, num_classes=len(model_dict))

    input_size = X_meta.shape[1]
    self.metadata_model = self._build_model(output_size=len(model_dict), input_size=input_size)
    self.metadata_model.fit(X_meta, y_meta, epochs=self.epochs, batch_size=self.batch_size, verbose=0)

  # Simple feedforward neural network to predict best model
  def _build_model(self, output_size, input_size=None):
    if input_size is None:
        raise ValueError("input_size must be provided to build metadata model.")

    model = keras.Sequential([
        keras.layers.Input(shape=(input_size,)),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(output_size, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

  # Check for linearly separable data (PCA + Logistic Regression)
  def _is_linearly_separable(self, X, y, threshold=0.8):
    try:
      pca = PCA(n_components=2)
      X_reduced = pca.fit_transform(X)
      clf = LogisticRegression().fit(X_reduced, y)
      return clf.score(X_reduced, y) > threshold
    except Exception as e:
      print(f"Linear separable check failed: {e}")
      return False

  # Check for linear regression trend (PCA + Ridge Regression)
  def _has_linear_trend(self, X, y, threshold=0.8):
    try:
      pca = PCA(n_components=1)
      X_reduced = pca.fit_transform(X)
      ridge = Ridge()
      ridge.fit(X_reduced, y)
      r2_score = ridge.score(X_reduced, y)
      return r2_score > threshold
    except Exception as e:
      print(f"Linear trend check failed: {e}")
      return False

  # Decide problem type (Classification or Regression)
  def _infer_problem_type(self, y):
    y = np.array(y)

    if all(isinstance(item, str) for item in y):
      return 'classifiers'

    if len(np.unique(y)) <= 20 and (np.issubdtype(y.dtype, np.integer) or np.allclose(y, np.round(y))):
      return 'classifiers'

    elif np.issubdtype(y.dtype, np.integer) or not np.issubdtype(y.dtype, np.number):
      return 'classifiers'

    return 'regressors'

  # Extract meta features from dataset
  def _extract_metadata(self, X, y):
    y = np.array(y)
    features = [
      X.shape[0],
      X.shape[1],
      len(np.unique(y)),
      np.mean(X),
      np.std(X),
      np.min(X),
      np.max(X),
      np.median(X),
      skew(X.flatten()),
      kurtosis(X.flatten()),
      1 if np.issubdtype(y.dtype, np.floating) else 0,
      1 if np.issubdtype(y.dtype, np.integer) else 0
    ]
    return np.array(features).reshape(-1)

  # Extract text features from the sequence of texts/strings
  def _extract_text_features(self, texts, max_len=100):
    tokenizer = keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    padded = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_len)

    vocab_size = len(tokenizer.word_index) + 1
    model = keras.Sequential([
        keras.layers.Embedding(input_dim=vocab_size, output_dim=64, input_length=max_len),
        keras.layers.GlobalAveragePooling1D()
    ])
    features = model.predict(padded, verbose=0)
    return features

  # Choose suitable evaluation metric
  def _select_metrics(self, y):
    if self._infer_problem_type(y) == 'classifiers':
      return 'accuracy'
    elif np.issubdtype(y.dtype, np.floating):
      return 'r2'
    else:
      return 'neg_mean_absolute_error'

  # Recommend based on data type
  def recommend(self, data):
    if self.data_type == 'image':
      return self._recommend_image(data)
    elif self.data_type == 'pixel':
      return self._recommend_pixel(data)
    elif self.data_type == 'tabular':
      return self._recommend_tabular(data)
    elif self.data_type == 'text':
      return self._recommend_text(data)
    else:
        raise ValueError("Invalid data type. Supported types are 'image', 'pixel', 'tabular', and 'text'.")

  # Recommendation function if data type is 'image'
  def _recommend_image(self, data):
    X, y = data
    X = X.astype('float32') / 255.0
    problem_type = self._infer_problem_type(y)
    model_dict = self.ml_models[problem_type]
    scoring_metric = self._select_metrics(y)

    input_shape = X.shape[1:]
    feature_extractor = keras.Sequential([
      keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
      keras.layers.MaxPooling2D(pool_size=(2, 2)),
      keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
      keras.layers.MaxPooling2D(pool_size=(2, 2)),
      keras.layers.Flatten(),
    ])

    X_features = feature_extractor.predict(X)

    if problem_type == 'classifiers' and self._is_linearly_separable(X_features, y):
      model = model_dict['Logistic Regression']
      score = cross_val_score(model, X_features, y, cv=5, scoring=scoring_metric)
      return 'classifiers', 'Logistic Regression', scoring_metric, float(np.mean(score))

    if self.metadata_model is None:
      raise ValueError("Model has not been trained. Please call `.fit()` first.")

    metadata = self._extract_metadata(X_features, y).reshape(1, -1)
    prediction = self.metadata_model.predict(metadata.reshape(1, -1)).flatten()
    best_model_idx = np.argmax(prediction)
    best_model_name = list(model_dict.keys())[best_model_idx]
    best_model = model_dict[best_model_name]

    scores = cross_val_score(best_model, X_features, y, cv=5, scoring=scoring_metric)
    return problem_type, best_model_name, scoring_metric, float(np.mean(scores))

  # Recommendation function if data type is 'pixel'
  def _recommend_pixel(self, data):
    X, y = data
    X = X.astype('float32') / 255.0
    problem_type = self._infer_problem_type(y)
    model_dict = self.ml_models[problem_type]
    scoring_metric = self._select_metrics(y)

    X_flat = X.reshape(X.shape[0], -1)
    X_flat = self.scaler.fit_transform(X_flat)

    if problem_type == 'classifiers' and self._is_linearly_separable(X_flat, y):
      model = model_dict['Logistic Regression']
      score = cross_val_score(model, X_flat, y, cv=5, scoring=scoring_metric)
      return 'classifiers', 'Logistic Regression', scoring_metric, float(np.mean(score))

    if problem_type == 'regressors' and self._has_linear_trend(X_flat, y):
      model = model_dict['Ridge Regression']
      score = cross_val_score(model, X_flat, y, cv=5, scoring=scoring_metric)
      return 'regressors', 'Ridge Regression', scoring_metric, float(np.mean(score))

    if self.metadata_model is None:
      raise ValueError("Model has not been trained. Please call `.fit()` first.")

    metadata = self._extract_metadata(X_flat, y).reshape(1, -1)
    prediction = self.metadata_model.predict(metadata).flatten()
    best_model_idx = np.argmax(prediction)
    best_model_name = list(model_dict.keys())[best_model_idx]
    best_model = model_dict[best_model_name]

    scores = cross_val_score(best_model, X_flat, y, cv=5, scoring=scoring_metric)
    return problem_type, best_model_name, scoring_metric, float(np.mean(scores))

  # Recommendation function if data type is 'tabular'
  def _recommend_tabular(self, data):
    if hasattr(data, "iloc"):
      X, y = data.iloc[:, :-1].values, data.iloc[:, -1].values

    elif isinstance(data, (tuple, list)) and len(data) == 2:
      X, y = data

    else:
      data = np.array(data)
      X, y = data[:, :-1], data[:, -1]

    X = self.scaler.fit_transform(X)

    metadata = self._extract_metadata(X, y).reshape(1, -1)

    problem_type = self._infer_problem_type(y)
    model_dict = self.ml_models[problem_type]
    scoring_metric = self._select_metrics(y)

    if problem_type == 'classifiers' and self._is_linearly_separable(X, y):
      if 'Logistic Regression' in model_dict:
        model = model_dict['Logistic Regression']
        score = cross_val_score(model, X, y, cv=5, scoring=scoring_metric)
        metric_value = -np.mean(score) if scoring_metric.startswith('neg_') else np.mean(score)
        return 'classifiers', 'Logistic Regression', scoring_metric, float(metric_value)

    if problem_type == 'regressors' and self._has_linear_trend(X, y):
      if 'Ridge Regression' in model_dict:
        model = model_dict['Ridge Regression']
        score = cross_val_score(model, X, y, cv=5, scoring=scoring_metric)
        metric_value = -np.mean(score) if scoring_metric.startswith('neg_') else np.mean(score)
        return 'regressors', 'Ridge Regression', scoring_metric, float(metric_value)

    if self.metadata_model is None:
      raise ValueError("Model has not been trained. Please call `.fit()` first.")

    prediction = self.metadata_model.predict(metadata.reshape(1, -1)).flatten()
    best_model_idx = np.argmax(prediction)
    best_model_name = list(model_dict.keys())[best_model_idx]
    best_model = model_dict[best_model_name]

    scores = cross_val_score(best_model, X, y, cv=5, scoring=scoring_metric)
    metric_value = -np.mean(scores) if scoring_metric.startswith('neg_') else np.mean(scores)

    return problem_type, best_model_name, scoring_metric, float(metric_value)

  # Recommendation function if data type is 'text'
  def _recommend_text(self, data):
    X_raw, y = data

    if not hasattr(self, 'vectorizer'):
      raise ValueError("Text vectorizer not found. Please call `.fit()` first.")

    X = self.vectorizer.transform(X_raw).toarray()
    X = self.scaler.transform(X)

    problem_type = self._infer_problem_type(y)
    model_dict = self.ml_models[problem_type]
    scoring_metric = self._select_metrics(y)

    if problem_type == 'classifiers' and self._is_linearly_separable(X, y):
      model = model_dict['Logistic Regression']
      score = cross_val_score(model, X, y, cv=5, scoring=scoring_metric)
      return 'classifiers', 'Logistic Regression', scoring_metric, float(np.mean(score))

    if problem_type == 'regressors' and self._has_linear_trend(X, y):
      model = model_dict['Ridge Regression']
      score = cross_val_score(model, X, y, cv=5, scoring=scoring_metric)
      return 'regressors', 'Ridge Regression', scoring_metric, float(np.mean(score))

    if self.metadata_model is None:
      raise ValueError("Model has not been trained. Please call `.fit()` first.")

    metadata = self._extract_metadata(X, y).reshape(1, -1)
    prediction = self.metadata_model.predict(metadata).flatten()
    best_model_idx = np.argmax(prediction)
    best_model_name = list(model_dict.keys())[best_model_idx]
    best_model = model_dict[best_model_name]


    scores = cross_val_score(best_model, X, y, cv=5, scoring=scoring_metric)
    return problem_type, best_model_name, scoring_metric, float(np.mean(scores))

  # Save the trained metadata neural model
  def save_model(self, filepath='metadata_model.h5'):
    if self.metadata_model:
        self.metadata_model.save(filepath)

  # Load the saved neural recommender
  def load_model(self, filepath='metadata_model.h5'):
      self.metadata_model = keras.models.load_model(filepath)

  # Evaluate the overall performance of the neural model
  def evaluate_recommendation_accuracy(self, data):
    if self.metadata_model is None:
        raise ValueError("Please train the model using `.fit()` before evaluation.")

    if self.data_type == 'tabular':
        if hasattr(data, "iloc"):
            X, y = data.iloc[:, :-1].values, data.iloc[:, -1].values
        elif isinstance(data, (tuple, list)) and len(data) == 2:
            X, y = data
        else:
            data = np.array(data)
            X, y = data[:, :-1], data[:, -1]

        X = self.scaler.fit_transform(X)

    elif self.data_type == 'pixel':
        X, y = data
        X = X.astype('float32') / 255.0
        X = X.reshape(X.shape[0], -1)
        X = self.scaler.fit_transform(X)

    elif self.data_type == 'text':
        X_raw, y = data
        X = self.vectorizer.transform(X_raw).toarray()
        X = self.scaler.transform(X)

    elif self.data_type == 'image':
        X, y = data
        X = X.astype('float32') / 255.0
        input_shape = X.shape[1:]
        feature_extractor = keras.Sequential([
            keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),
            keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),
            keras.layers.Flatten(),
        ])
        X = feature_extractor.predict(X, verbose=0)
        pca = PCA(n_components=min(50, X.shape[1]))
        X = pca.fit_transform(X)

    else:
        raise ValueError("Unsupported data_type for evaluation.")

    y = self.label_encoder.fit_transform(y)
    metadata = self._extract_metadata(X, y).reshape(1, -1)
    problem_type = self._infer_problem_type(y)
    model_dict = self.ml_models[problem_type]
    scoring_metric = self._select_metrics(y)

    # Get actual model performances
    model_scores = {}
    for model_name, model in model_dict.items():
        try:
            score = cross_val_score(model, X, y, cv=3, scoring=scoring_metric)
            model_scores[model_name] = np.mean(score)
        except Exception:
            continue

    if not model_scores:
        return 0.0

    # Normalize model scores
    scores = np.array(list(model_scores.values()))
    min_score = scores.min()
    max_score = scores.max()
    if max_score == min_score:
        norm_scores = {k: 1.0 for k in model_scores}  # If all models same score
    else:
        norm_scores = {k: (v - min_score) / (max_score - min_score) for k, v in model_scores.items()}

    # Predicted best model by metadata model
    prediction = self.metadata_model.predict(metadata, verbose=0).flatten()
    predicted_model_idx = np.argmax(prediction)
    predicted_model_name = list(model_dict.keys())[predicted_model_idx]

    return norm_scores.get(predicted_model_name, 0.0)

# **Training and Testing**

## Train on Pixel Dataset


In [ ]:
from tensorflow.keras.datasets import mnist

# Load MNIST data
(X_train, y_train), (_, _) = mnist.load_data()

# Reshape to include channel dimension
X_train = X_train.reshape(-1, 28, 28, 1)

# Take a small sample
X_sample = X_train[:1000]
y_sample = y_train[:1000]

# Instantiate and train the recommender
recommender = NeuralNetworkModelRecommender(data_type='pixel')
recommender.fit((X_sample, y_sample))

# Get model recommendation
result = recommender.recommend((X_sample, y_sample))
print("\nRecommendation Result:")
print(f"Problem Type: {result[0]}")
print(f"Model Name: {result[1]}")
print(f"Scoring Metric: {result[2]}")
print(f"Score: {result[3]:.4f}")

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Evaluate
accuracy_score = recommender.evaluate_recommendation_accuracy((
    X_sample[:200],
    y_sample[:200]
))
print(f"Recommendation Accuracy (probabilistic): {accuracy_score:.4f}")

## Train on Text Dataset

In [ ]:
# Prepare text data and labels
texts = [
    "I love this product!",
    "This is the worst thing ever.",
    "Amazing experience, would buy again.",
    "Terrible customer service.",
    "Totally worth the price.",
    "It broke after one use.",
    "Best purchase I’ve made this year.",
    "I hate it so much.",
    "Very satisfied with the results.",
    "Not recommended at all."
]
labels = [
    "positive",
    "negative",
    "positive",
    "negative",
    "positive",
    "negative",
    "positive",
    "negative",
    "positive",
    "negative"
]

# Instantiate and train the recommender
recommender = NeuralNetworkModelRecommender(data_type='text', epochs=10)
recommender.fit((texts, labels))

# Recommend the best ML model based on metadata
result = recommender.recommend((texts, labels))
print("\nRecommendation Result:")
print(f"Problem Type: {result[0]}")
print(f"Best Model: {result[1]}")
print(f"Scoring Metric: {result[2]}")
print(f"Cross-Validation Score: {result[3]:.4f}")

In [ ]:
# Evaluate
accuracy_score = recommender.evaluate_recommendation_accuracy((texts, labels))
print(f"Recommendation Accuracy (probabilistic): {accuracy_score:.4f}")

## Train on Image Dataset

In [ ]:
from sklearn.datasets import load_digits
from skimage.transform import resize

# Load original data
digits = load_digits()
X, y = digits.images, digits.target

# Resize to 32x32 and reshape
X_resized = np.array([resize(img, (32, 32), anti_aliasing=True) for img in X])
X_resized = X_resized.reshape(-1, 32, 32, 1)

# Verify the size of image
print(f"Image dataset resized: {X_resized.shape[0]} samples of shape {X_resized.shape[1:]}")

# Train as before
recommender = NeuralNetworkModelRecommender(data_type='image', epochs=20)
recommender.fit((X_resized, y))

# Run recommendation
problem_type, best_model, metric, score = recommender.recommend((X_resized, y))
print("\nRecommendation Result:")
print(f"Problem Type: {problem_type}")
print(f"Best Model: {best_model}")
print(f"Scoring Metric: {metric}")
print(f"Cross-Validation Score: {score:.4f}")

## Train on Tabular Data (Classification)

In [ ]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

# Load the tabular dataset
data = load_iris()
X, y = data.data, data.target

# Optional: Scale features for better training stability
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize and train the recommender for tabular data
recommender = NeuralNetworkModelRecommender(data_type='tabular', epochs=20)
recommender.fit((X_train, y_train))

# Run recommendation
problem_type, best_model, metric, score = recommender.recommend((X_test, y_test))
print("Recommendation Result:")
print(f"Problem Type: {problem_type}")
print(f"Best Model: {best_model}")
print(f"Scoring Metric: {metric}")
print(f"Cross-Validation Score: {score:.4f}")

In [ ]:
# Evaluate
accuracy_score = recommender.evaluate_recommendation_accuracy((X_test, y_test))
print(f"Recommendation Accuracy (probabilistic): {accuracy_score:.4f}")

## Train on Tabular data (Regression)

In [ ]:
from sklearn.datasets import fetch_california_housing

# Load regression dataset
data = fetch_california_housing()
X, y = data.data, data.target

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
sample_indices = np.random.choice(len(X_train), size=1000, replace=False)
X_sample = X_train[sample_indices]
y_sample = y_train[sample_indices]

# Initialize and train the recommender for regression
recommender = NeuralNetworkModelRecommender(data_type='tabular', epochs=5)
recommender.fit((X_sample, y_sample))

# Run recommendation
problem_type, best_model, metric, score = recommender.recommend((X_test, y_test))
print("\nRecommendation Result:")
print(f"Problem Type: {problem_type}")
print(f"Best Model: {best_model}")
print(f"Scoring Metric: {metric}")
print(f"Score: {score:.4f}")

# **Notes**
# Advantages
- Automated scaling, normalization, feature extraction (using PCA, CNN-based feature extraction for images), and text vectorization are integrated into the pipeline
- Multi-modal support for tabular, image, pixel, and text data.
- Includes heuristic methods such as linear separability and linear trend checks, saving computational resources
- Extracts key statistical metadata (e.g., mean, std, skewness, kurtosis) from the data and uses a neural network to predict the best performing algorithm
- Diverse Algorithm Options
- Cross-Validation for Robust Estimation
- Clean Code Structure
- Fast for Classification Problems

# Disadvantages
- Limited Generalization due to high dependence on heuristic methods
- Overhead in Feature Extraction for Images
- Limited Feature Insights
- Fixed Underlying Model Parameters (e.g., RandomForestClassifier, GradientBoostingClassifier)
- Potential Overfitting of the Metadata Model
- Inconsistent Accuracy for Smaller Dataset (For  the Overall Model not Individual  Models)
- Slow for Regression Problem Type